# ファイルのアップロード方法

## Google Driveにファイルをアップロード

Google Driveに`SSL`フォルダを作成し、その中に以下のファイル/フォルダをアップロードしてください：

1. **task/** フォルダ（全体）
   - `__init__.py`
   - `dataset.py`
   - `model.py`
   - `train.py`

2. **noise/** フォルダ（全体）
   - `__init__.py`
   - `add_noise.py`
   - `frequency_band_noise.py`
   - `localized_spike_noise.py`
   - `amplitude_dependent_noise.py`

3. **eval.py** ファイル

4. **data_lowF_noise.pickle** ファイル

**フォルダ構造**:
```
Google Drive/
└── MyDrive/
    └── SSL/
        ├── task/
        ├── noise/
        ├── eval.py
        └── data_lowF_noise.pickle
```



In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')


# タスク4: マスク予測 + 正則化項

自己教師あり学習によるノイズ検出


## 1. 環境設定


In [ ]:
# このセルは不要（セル1でマウント済み）


In [ ]:
# 必要なライブラリをインストール
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install scikit-learn matplotlib


In [ ]:
# 作業ディレクトリを設定
import os
import sys

# Google DriveのSSLフォルダを作業ディレクトリに設定
project_path = '/content/drive/MyDrive/SSL'
if os.path.exists(project_path):
    os.chdir(project_path)
    sys.path.append(project_path)
    print(f"Working directory: {os.getcwd()}")
else:
    print(f"⚠️ {project_path} が見つかりません。")
    print("Google DriveにSSLフォルダを作成して、ファイルをアップロードしてください。")


## 2. データの確認


In [ ]:
# データファイルのパスを設定
pickle_path = '/content/drive/MyDrive/SSL/data_lowF_noise.pickle'

# データが存在するか確認
import os
if os.path.exists(pickle_path):
    print(f"✅ Data file found: {pickle_path}")
else:
    print(f"⚠️ Data file not found at {pickle_path}")
    print("Google DriveのSSLフォルダにdata_lowF_noise.pickleをアップロードしてください。")


## 3. モデルのインポート


In [ ]:
# ファイルの存在確認
import os

# 必要なファイル/フォルダの存在確認
base_path = '/content/drive/MyDrive/SSL'
required_files = [
    f'{base_path}/task/__init__.py',
    f'{base_path}/task/dataset.py',
    f'{base_path}/task/model.py',
    f'{base_path}/task/train.py',
    f'{base_path}/noise/__init__.py',
    f'{base_path}/noise/add_noise.py',
    f'{base_path}/noise/frequency_band_noise.py',
    f'{base_path}/noise/localized_spike_noise.py',
    f'{base_path}/noise/amplitude_dependent_noise.py',
    f'{base_path}/eval.py',
    f'{base_path}/data_lowF_noise.pickle'
]

missing_files = []
for file in required_files:
    if not os.path.exists(file):
        missing_files.append(file.replace(base_path + '/', ''))

if missing_files:
    print("⚠️ 以下のファイル/フォルダが見つかりません:")
    for file in missing_files:
        print(f"  - {file}")
    print(f"\n📁 Google Driveの {base_path} にファイルをアップロードしてください。")
    print("   フォルダ構造:")
    print("   SSL/")
    print("   ├── task/")
    print("   ├── noise/")
    print("   ├── eval.py")
    print("   └── data_lowF_noise.pickle")
else:
    print("✅ すべての必要なファイルが見つかりました！")

# タスク4のモジュールをインポート
try:
    from task.dataset import Task4Dataset
    from task.model import Task4BERT
    from task.train import train_task4
    import eval
    print("✅ モジュールのインポートに成功しました！")
except ImportError as e:
    print(f"❌ モジュールのインポートに失敗しました: {e}")
    print("   ファイルが正しくアップロードされているか確認してください。")


## 4. 学習の実行


In [ ]:
# デバイスの確認
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")


In [ ]:
# チェックポイントを削除して最初から学習する場合（必要に応じて実行）
import os
checkpoint_path = "/content/drive/MyDrive/SSL/task4_output/checkpoint.pth"
if os.path.exists(checkpoint_path):
    print("既存のチェックポイントが見つかりました。削除します...")
    os.remove(checkpoint_path)
    print("✓ チェックポイントを削除しました。最初から学習します。")
else:
    print("チェックポイントは存在しません。最初から学習します。")


In [ ]:
# 学習を実行
# 最初から学習する場合は resume=False に変更してください
# チェックポイントから再開する場合は resume=True のままにしてください
model, train_losses, val_losses, best_val_loss = train_task4(
    pickle_path=pickle_path,
    batch_size=16,  # 速度向上のため8→16に変更
    num_epochs=10,  # エポック数（10エポックで約1時間）
    lr=1e-4,
    val_ratio=0.2,
    device=device,
    out_dir="/content/drive/MyDrive/SSL/task4_output",  # 出力先をGoogle Driveに
    resume=False,  # Falseに変更: 最初から学習 / True: チェックポイントから再開
    lambda_reg=0.5,  # 正則化項の重みを増やす（0.1 → 0.5）
    num_intervals=30,
    noise_type='frequency_band',  # ノイズタイプ: 'frequency_band', 'localized_spike', 'amplitude_dependent'
    noise_level=0.3,  # ノイズレベル（30%）
    margin=0.2,  # ランキング損失のマージンを増やす（0.1 → 0.2）
)


In [ ]:
# 評価に必要なモジュールをインポート
import os
import sys
import importlib

# 作業ディレクトリを設定（重要！）
project_path = '/content/drive/MyDrive/SSL'
if os.path.exists(project_path):
    os.chdir(project_path)
    sys.path.insert(0, project_path)
    print(f"Working directory: {os.getcwd()}")
else:
    print(f"⚠️ {project_path} が見つかりません。")
    print("Google DriveにSSLフォルダを作成して、ファイルをアップロードしてください。")

# 必要なモジュールをインポート
from task.dataset import Task4Dataset
from task.model import Task4BERT
import torch
import eval

# evalモジュールを再読み込み（eval.pyを変更した場合）
if 'eval' in sys.modules:
    importlib.reload(sys.modules['eval'])
    print("evalモジュールを再読み込みしました")

# デバイスの設定（まだ設定されていない場合）
if 'device' not in globals():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

# データパスの設定（まだ設定されていない場合）
if 'pickle_path' not in globals():
    pickle_path = "/content/drive/MyDrive/SSL/data_lowF_noise.pickle"

# モデルの初期化（まだ初期化されていない場合）
if 'model' not in globals() or model is None:
    seq_len = 3000
    model = Task4BERT(
        seq_len=seq_len,
        d_model=64,
        n_heads=2,
        num_layers=2,
        dim_feedforward=128,
        dropout=0.1,
    ).to(device)
    print("モデルを初期化しました")


## 5. 評価の実行


In [ ]:
# テストデータで評価
from torch.utils.data import DataLoader

# テストデータセットを作成
test_dataset = Task4Dataset(
    pickle_path=pickle_path,
    num_intervals=30,
    noise_type='frequency_band',  # 学習時と同じノイズタイプ
    noise_level=0.3,  # 学習時と同じノイズレベル
    add_structured_noise_flag=True,  # 全体的な構造化ノイズを付与（学習時と同じ）
)

# テストデータローダーを作成（全データを使用）
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)  # 速度向上のため8→16に変更

# 最良のモデルを読み込み
best_model_path = "/content/drive/MyDrive/SSL/task4_output/best_val_model.pth"
model.load_state_dict(torch.load(best_model_path, map_location=device))

# 評価を実行
results = eval.evaluate_model(
    model=model,
    dataloader=test_loader,
    method='self_supervised',
    device=device,
    num_intervals=30,
)

# 結果を表示
print("\n評価結果:")
print(f"Accuracy: {results['accuracy']:.4f}")
print(f"Precision: {results['precision']:.4f}")
print(f"Recall: {results['recall']:.4f}")
print(f"F1-score: {results['f1_score']:.4f}")
if 'roc_auc' in results:
    print(f"ROC-AUC: {results['roc_auc']:.4f}")
if 'attention_diff' in results:
    print(f"アテンションウェイトの差: {results['attention_diff']:.4f}")
    print(f"  (正常区間 - ノイズ区間)")
if 'best_threshold' in results:
    print(f"最適閾値: {results['best_threshold']:.6f}")
if 'noise_attention_mean' in results:
    print(f"ノイズ区間の平均アテンション: {results['noise_attention_mean']:.6f}")
if 'normal_attention_mean' in results:
    print(f"正常区間の平均アテンション: {results['normal_attention_mean']:.6f}")


In [ ]:
# 混同行列を可視化
eval.plot_confusion_matrix(
    results['confusion_matrix'],
    title='タスク4: 混同行列',
    save_path="/content/drive/MyDrive/SSL/task4_output/confusion_matrix.png"
)
print("混同行列を保存しました")


In [ ]:
# アテンションウェイトの分布を可視化
if 'attention_weights' in results:
    eval.plot_attention_distribution(
        results['attention_weights'],
        results['labels'],
        num_intervals=30,
        save_path="/content/drive/MyDrive/SSL/task4_output/attention_distribution.png"
    )
    print("アテンションウェイトの分布を保存しました")
